<!--NOTEBOOK_HEADER-->
*This notebook contains material from [PyRosetta](https://RosettaCommons.github.io/PyRosetta.notebooks);
content is available [on Github](https://github.com/RosettaCommons/PyRosetta.notebooks.git).*

<!--NAVIGATION-->
< [Packing and Relax](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.02-Packing-design-and-regional-relax.ipynb) | [Contents](toc.ipynb) | [Index](index.ipynb) | [Protein Design 2](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.04-Protein-Design-2.ipynb) ><p><a href="https://colab.research.google.com/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.03-Design-with-a-resfile-and-relax.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>

# Protein Design with a Resfile and FastRelax

Keywords: FastDesign, FastRelax, ResfileCommandOperation, Resfile, ResidueSelector, MoveMapFactory, TaskFactory, TaskOperation, NoRepackDisulfides, IncludeCurrent, ReadResfile, conf2pdb_chain(), pose_from_rcsb(), create_score_function(), CA_rmsd()

## Overview

In this Workshop, we will learn the classic way to design proteins, but in the same breath introduce the concept of design using a flexible backbone protocol.  

This protocol, is essentially design during FastRelax.  A separate class, FastDesign, has a bit more options for design, but essentially, they are the same.  

Many modern designs have used this FastDesign/RelaxedDesign protocol - including many Science papers from the Baker lab and the RosettaAntibodyDesign (RAbD) protocol that we will cover in another tutorial. 

Before this workshop, you should read about the resfile syntax here: https://www.rosettacommons.org/docs/latest/rosetta_basics/file_types/resfiles

*Warning*: This notebook uses `pyrosetta.distributed.viewer` code, which runs in `jupyter notebook` and might not run if you're using `jupyterlab`.

In [ ]:
# Notebook setup
import sys
if 'google.colab' in sys.modules:
    !pip install pyrosettacolabsetup
    import pyrosettacolabsetup
    pyrosettacolabsetup.setup()
    print ("Notebook is set for PyRosetta use in Colab.  Have fun!")

**Make sure you are in the directory with the pdb files:**

`cd google_drive/My\ Drive/student-notebooks/`

In [2]:
import logging
logging.basicConfig(level=logging.INFO)
import pyrosetta
import pyrosetta.toolbox

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Initialize PyRosetta

In [3]:
pyrosetta.init("-ignore_unrecognized_res 1 -ex1 -ex2aro -detect_disulf 0")

INFO:pyrosetta.rosetta:Found rosetta database at: /Users/jadolfbr/Library/Python/2.7/lib/python/site-packages/pyrosetta-2019.33+release.1e60c63beb5-py2.7-macosx-10.14-intel.egg/pyrosetta/database; using it....
INFO:pyrosetta.rosetta:PyRosetta-4 2019 [Rosetta PyRosetta4.Release.python27.mac 2019.33+release.1e60c63beb532fd475f0f704d68d462b8af2a977 2019-08-09T15:19:57] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
INFO:rosetta:core.init: Checking for fconfig files in pwd and ./rosetta/flags
INFO:rosetta:core.init: Reading fconfig.../Users/jadolfbr/.rosetta/flags/common
INFO:rosetta:core.init: 
INFO:rosetta:core.init: 
INFO:rosetta:core.init: Rosetta version: PyRosetta4.Release.python27.mac r230 2019.33+release.1e60c63beb5 1e60c63beb532fd475f0f704d68d462b8af2a977 http://www.pyrosetta.org 2019-08-09T15:19:57
INFO:rosetta:core.init: command: PyRosetta -ignore_unrecognized_res 1 -ex1 -ex2aro -det

PyRosetta-4 2019 [Rosetta PyRosetta4.Release.python27.mac 2019.33+release.1e60c63beb532fd475f0f704d68d462b8af2a977 2019-08-09T15:19:57] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


 For this tutorial, let's use the well-studied native protein crambin from PDB ID 1AB1 (http://www.rcsb.org/structure/1AB1).
 
 Setup the input pose and scorefunction:

In [4]:
start_pose = pyrosetta.toolbox.rcsb.pose_from_rcsb("1AB1", ATOM=True, CRYS=False)
pose = start_pose.clone()
scorefxn = pyrosetta.create_score_function("ref2015_cart.wts")

INFO:rosetta:core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 980 residue types
INFO:rosetta:core.chemical.GlobalResidueTypeSet: Total time to initialize 0.915967 seconds.
INFO:rosetta:core.import_pose.import_pose: File '1AB1.clean.pdb' automatically determined to be of type PDB
INFO:rosetta:core.io.pdb.pdb_reader: Parsing 0 .pdb records with unknown format to search for Rosetta-specific comments.
INFO:rosetta:core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 3 atoms at position 1 in file 1AB1.clean.pdb. Best match rsd_type:  THR:NtermProteinFull
INFO:rosetta:core.scoring.etable: Starting energy table calculation
INFO:rosetta:core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
INFO:rosetta:core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
INFO:rosetta:core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
INFO:rosetta:core.scoring.etabl

 Make of list of which residues are cysteine:

In [5]:
cys_res = []
for i, aa in enumerate(start_pose.sequence(), start=1):
    if aa == "C":
        cys_res.append(i)
print(cys_res)

[3, 4, 16, 26, 32, 40]


Inspect `start_pose` using the `PyMolMover` or `dump_pdb()`

## Design strategy:

Design away the cysteine residues (i.e. disulfide bonds) using a resfile, allowing all side-chains to re-pack and all backbone and side-chain torsions to minimize using the `FastRelax` mover.

 Read more about resfile file structure at https://www.rosettacommons.org/docs/latest/rosetta_basics/file_types/resfiles

To write a resfile, we need to know which chain to mutate.

We can see that the pose consists of only chain "A" by printing the `pose.pdb_info()` object:

In [6]:
print(pose.pdb_info())

PDB file name: 1AB1.clean.pdb
 Pose Range  Chain    PDB Range  |   #Residues         #Atoms

0001 -- 0046    A 0001  -- 0046  |   0046 residues;    00649 atoms
                           TOTAL |   0046 residues;    00649 atoms



 More programmatically, we could find which chains are in the `pose` using `pyrosetta.rosetta.core.pose.conf2pdb_chain(pose)` which returns a `pyrosetta.rosetta.std.map_unsigned_long_char` object which is iterable.

In [7]:
print(pyrosetta.rosetta.core.pose.conf2pdb_chain(pose))

map_unsigned_long_char{1: A}


In [8]:
for k, v in pyrosetta.rosetta.core.pose.conf2pdb_chain(pose).items():
    print(v)

A


 So we could write a resfile to disc indicating design specifications to mutate only the cysteine residues in chain "A".  Use the syntax described below, and save your resfile in this directory as `resfile`.
 
 https://www.rosettacommons.org/docs/latest/rosetta_basics/file_types/resfiles

In [9]:
### BEGIN SOLUTION
resfile = "./outputs/resfile"
with open(resfile, "w") as f:
    f.write("NATAA\n")
    f.write("start\n")
    for i in cys_res:
        f.write("{0} {1} ALLAAxc\n".format(i, pyrosetta.rosetta.core.pose.conf2pdb_chain(pose)[1]))
!cat {resfile}

### END SOLUTION

NATAA
start
3 A ALLAAxc
4 A ALLAAxc
16 A ALLAAxc
26 A ALLAAxc
32 A ALLAAxc
40 A ALLAAxc


Note that we don't necessarily need a resfile to use resfile commands.  We can now do this in an intuitive way through code and `ResidueSelectors` using the `ResfileCommandOperation`.  The main docs for the XML interface are available below, however the code-level interface is extremely similar.  Use the ? to get more info on this.  The operation is located in `pyrosetta.rosetta.core.pack.task.operation` as we saw this location in the previous tutorial.

https://www.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/TaskOperations/taskoperations_pages/ResfileCommandOperation

 Now we can setup the TaskOperations for the `FastRelax` mover. These tell `FastRelax` which residues to design or repack during the packer steps in `FastRelax`.  You should be familiar with this from the previous tutorial
 
We use the `IncludeCurrent` to include the current rotamer of from the crystal structure during packing.

In [10]:
# The task factory accepts all the task operations
tf = pyrosetta.rosetta.core.pack.task.TaskFactory()

# These are pretty standard
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.InitializeFromCommandline())
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.IncludeCurrent())
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.NoRepackDisulfides())

# Include the resfile
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.ReadResfile(resfile))

# Convert the task factory into a PackerTask to take a look at it
packer_task = tf.create_task_and_apply_taskoperations(pose)
# View the PackerTask
print(packer_task)

INFO:rosetta:core.pack.task: Packer task: initialize from command line()


#Packer_Task

resid	pack?	design?	allowed_aas
1	TRUE	FALSE	THR:NtermProteinFull
2	TRUE	FALSE	THR
3	TRUE	TRUE	ALA,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
4	TRUE	TRUE	ALA,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
5	TRUE	FALSE	PRO
6	TRUE	FALSE	SER
7	TRUE	FALSE	ILE
8	TRUE	FALSE	VAL
9	TRUE	FALSE	ALA
10	TRUE	FALSE	ARG
11	TRUE	FALSE	SER
12	TRUE	FALSE	ASN
13	TRUE	FALSE	PHE
14	TRUE	FALSE	ASN
15	TRUE	FALSE	VAL
16	TRUE	TRUE	ALA,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
17	TRUE	FALSE	ARG
18	TRUE	FALSE	LEU
19	TRUE	FALSE	PRO
20	TRUE	FALSE	GLY
21	TRUE	FALSE	THR
22	TRUE	FALSE	SER
23	TRUE	FALSE	GLU
24	TRUE	FALSE	ALA
25	TRUE	FALSE	ILE
26	TRUE	TRUE	ALA,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
27	TRUE	FALSE	ALA
28	TRUE	FALSE	THR
29	TRUE	FALSE	TYR
30	TRUE	FALSE	THR
31	TRUE	FALSE	GLY
32	TRUE	TRUE	ALA,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,T

 The PackerTask looks as intended! 

 Now we can set up a `MoveMap` or a `MoveMapFactory` to specify which torsions are free to minimize during the minimization steps of the `FastDesign` mover

In [11]:
# Set up a MoveMapFactory
mmf = pyrosetta.rosetta.core.select.movemap.MoveMapFactory()
mmf.all_bb(setting=True)
mmf.all_bondangles(setting=True)
mmf.all_bondlengths(setting=True)
mmf.all_chi(setting=True)
mmf.all_jumps(setting=True)
mmf.set_cartesian(setting=True)

In [12]:
# Set up a MoveMap
# mm = pyrosetta.rosetta.core.kinematics.MoveMap()
# mm.set_bb(True)
# mm.set_chi(True)
# mm.set_jump(True)

# If needed, you could turn off bb and chi torsions for individual residues like this:

# vector1 of true/false for each residue in the pose
# subset_to_minimize = do_something_set.apply(pose)

# for i in range(1, pose.size() + 1):
#     if (not subset_to_minimize[i]):
#         mm.set_bb(i, False)
#         mm.set_chi(i, False)

 Because some Movers only take as input a `MoveMap`, for backwards-compatibility one could generate a `MoveMap` from a `MoveMapFactory` using the `MoveMapFactory` function `create_movemap_from_pose(pose)`

 Now let's double-check some more `pose` information to verify that we are ready for `FastRelax`:

In [13]:
display_pose = pyrosetta.rosetta.protocols.fold_from_loops.movers.DisplayPoseLabelsMover()
display_pose.tasks(tf)
display_pose.movemap_factory(mmf)
display_pose.apply(pose)

INFO:rosetta:protocols.DsspMover: LEELLLHHHHHHHHHHHLLLLLHHHHHHHHLLEELLLLLLLLLLLL
INFO:rosetta:protocols.fold_from_loops.DisplayPoseLabelsMover: SEQUENCE       TTCCPSIVARSNFNVCRLPGTSEAICATYTGCIIIPGATCPGDYAN
INFO:rosetta:protocols.fold_from_loops.DisplayPoseLabelsMover: STRUCTURE      LEELLLHHHHHHHHHHHLLLLLHHHHHHHHLLEELLLLLLLLLLLL
INFO:rosetta:protocols.fold_from_loops.DisplayPoseLabelsMover: MVMP_BB        **********************************************
INFO:rosetta:protocols.fold_from_loops.DisplayPoseLabelsMover: MVMP_CHI       **********************************************
INFO:rosetta:core.pack.task: Packer task: initialize from command line()
INFO:rosetta:protocols.fold_from_loops.DisplayPoseLabelsMover: DESIGN         --**-----------*---------*-----*-------*------
INFO:rosetta:protocols.fold_from_loops.DisplayPoseLabelsMover: REPACK         **********************************************
INFO:rosetta:protocols.fold_from_loops.DisplayPoseLabelsMover: FOLDTREE       ******************

 Setting up `FastRelax` prints the default `relaxscript`, showing the `ramp_repack_min` settings with the following assignments:
>ramp_repack_min [scale:fa_rep] [min_tolerance] [coord_cst_weight]

In [15]:
fr = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxn_in=scorefxn, standard_repeats=1)
fr.cartesian(True)
fr.set_task_factory(tf)
fr.set_movemap_factory(mmf)
fr.min_type("lbfgs_armijo_nonmonotone") # For non-Cartesian scorefunctions, use "dfpmin_armijo_nonmonotone"

#Note that this min_type is automatically set when you set the cartesian option. 
#  But it is good to be aware of this - as not all protocols will do this for you.

#fr.set_movemap(mm) # Could have optionally specified a MoveMap instead of MoveMapFactory
#fr.minimize_bond_angles(True) # If not using MoveMapFactory, could specify bond angle minimization here
#fr.minimize_bond_lengths(True) # If not using MoveMapFactory, could specify bond length minimization here

INFO:rosetta:protocols.relax.RelaxScriptManager: Reading relax scripts list from database.
INFO:rosetta:protocols.relax.RelaxScriptManager: Looking for default.txt
INFO:rosetta:protocols.relax.RelaxScriptManager: ================== Reading script file: /Users/jadolfbr/Library/Python/2.7/lib/python/site-packages/pyrosetta-2019.33+release.1e60c63beb5-py2.7-macosx-10.14-intel.egg/pyrosetta/database/sampling/relax_scripts/default.txt ==================
INFO:rosetta:protocols.relax.RelaxScriptManager: repeat %%nrepeats%%
INFO:rosetta:protocols.relax.RelaxScriptManager: ramp_repack_min 0.02  0.01     1.0
INFO:rosetta:protocols.relax.RelaxScriptManager: ramp_repack_min 0.250 0.01     0.5
INFO:rosetta:protocols.relax.RelaxScriptManager: ramp_repack_min 0.550 0.01     0.0
INFO:rosetta:protocols.relax.RelaxScriptManager: ramp_repack_min 1     0.00001  0.0
INFO:rosetta:protocols.relax.RelaxScriptManager: accept_to_best
INFO:rosetta:protocols.relax.RelaxScriptManager: endrepeat


For recommendations on setting `fr.min_type()` for the scorefunction being used, see: https://www.rosettacommons.org/docs/latest/rosetta_basics/structural_concepts/minimization-overview#recommendations

 Run Fast(Design)! Note: this takes ~1min 31s

In [16]:
### BEGIN SOLUTION
%time fr.apply(pose)
### END SOLUTION

INFO:rosetta:core.scoring.CartesianBondedEnergy: Creating new peptide-bonded energy container (46)
INFO:rosetta:basic.io.database: Database file opened: scoring/score_functions/elec_cp_reps.dat
INFO:rosetta:core.scoring.elec.util: Read 40 countpair representative atoms
INFO:rosetta:core.pack.dunbrack.RotamerLibrary: shapovalov_lib_fixes_enable option is true.
INFO:rosetta:core.pack.dunbrack.RotamerLibrary: shapovalov_lib::shap_dun10_smooth_level of 1( aka lowest_smooth ) got activated.
INFO:rosetta:core.pack.dunbrack.RotamerLibrary: Binary rotamer library selected: /Users/jadolfbr/Library/Python/2.7/lib/python/site-packages/pyrosetta-2019.33+release.1e60c63beb5-py2.7-macosx-10.14-intel.egg/pyrosetta/database/rotamer/shapovalov/StpDwn_0-0-0/Dunbrack10.lib.bin
INFO:rosetta:core.pack.dunbrack.RotamerLibrary: Using Dunbrack library binary file '/Users/jadolfbr/Library/Python/2.7/lib/python/site-packages/pyrosetta-2019.33+release.1e60c63beb5-py2.7-macosx-10.14-intel.egg/pyrosetta/database/r

CPU times: user 2min 51s, sys: 1.02 s, total: 2min 52s
Wall time: 2min 52s


### Analysis

Inspect the resulting design!

 By how many Angstroms RMSD did the backbone Cα atoms move?

In [17]:
pyrosetta.rosetta.core.scoring.CA_rmsd(start_pose, pose)

0.798085629940033

What is the delta `total_score` from `start_pose` to `pose`? Why is it large?

In [18]:
delta_total_score = scorefxn(pose) - scorefxn(start_pose)
print(delta_total_score)

INFO:rosetta:core.scoring.CartesianBondedEnergy: Creating new peptide-bonded energy container (46)


-1377.51918428


 What is the per-residue energy difference for each mutated position between `start_pose` and `pose`?

In [19]:
### BEGIN SOLUTION

for i in cys_res:
    pose_total_score = pyrosetta.rosetta.protocols.relax.get_per_residue_scores(pose, pyrosetta.rosetta.core.scoring.ScoreType.total_score)[i - 1]
    start_pose_total_score = pyrosetta.rosetta.protocols.relax.get_per_residue_scores(start_pose, pyrosetta.rosetta.core.scoring.ScoreType.total_score)[i - 1]
    print("The delta total_score for residue {} between start_pose and pose is {}".format(i, pose_total_score - start_pose_total_score))
    
### END SOLUTION

The delta total_score for residue 3 between start_pose and pose is -12.881750115
The delta total_score for residue 4 between start_pose and pose is -162.36925048
The delta total_score for residue 16 between start_pose and pose is -34.5286984711
The delta total_score for residue 26 between start_pose and pose is -7.32796169396
The delta total_score for residue 32 between start_pose and pose is -6.70455775554
The delta total_score for residue 40 between start_pose and pose is -7.55635449526


<!--NAVIGATION-->
< [Packing and Relax](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.02-Packing-design-and-regional-relax.ipynb) | [Contents](toc.ipynb) | [Index](index.ipynb) | [Protein Design 2](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.04-Protein-Design-2.ipynb) ><p><a href="https://colab.research.google.com/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.03-Design-with-a-resfile-and-relax.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>